### Third try for widgets

See: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

and: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html

This notebook is a sandbox for exploring various techniques for developing widgets for design programs.

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

from Designer import SST,show
dshow = show

In [6]:
class What(object):
    
    def __init__(self):
        self.section = None
        self.data = {}
        self.widgets = {}
        self.properties = set()
        self.widget_list = []
        self.output_widget = None
        self.main_widget = None
        self.compute_button = None
        self.defaults = {}
        
    def setDefaults(self,**kwargs):
        self.defaults.update(kwargs)
    
    def _build(self):
        self.output_widget = widgets.Output(layout={'border': '1px solid black'})
        with self.output_widget:
            b = widgets.Button(description='Compute!',tooltip='Click to perform calculations')
            b.on_click(self._on_compute_clicked)
            self.compute_button = b
            allwidg = widgets.VBox(self.widget_list+[b])
            self.main_widget = allwidg
    
    def _on_compute_clicked(self,b):
        """Examine the arguments of the callable and drag them out of data."""
        with self.output_widget:
            for key,w in self.widgets.items():
                self.data[key] = w.value
            self.callable()
        
    def interact(self,callable):
        self._build()
        self.callable = callable
        self._updateWidgetValues()
        display(self.main_widget)
        display(self.output_widget)
        
    def _updateWidgetValues(self):
        pass
        
    def _floatWidgets(self,text):
        ans = []
        for key in text.split(','):
            key = key.strip()
            val = 0.
            if '=' in key:
                k,v = key.split('=',1)
                key = k.strip()
                val = float(eval(v))
                self.defaults[key] = val
            w = widgets.FloatText(value=val,description=key+':')
            ans.append((key,w))
        return ans
    
    def _on_load_clicked(self,b):
        pass
    
    def defWidgets(self,text,properties=False):
        wlist = []
        if properties:
            dwidg = widgets.Text(value='W250x80',
                               placeholder='Shape Designation',
                               description='Dsg:',
                              tooltip='A shape designation to lookup for values.')
            pbut = widgets.Button(description='Load Properties',
                                   tooltip='Click to load properties into entries below.')
            pbut.on_click(self._on_load_clicked)
            top = widgets.HBox([dwidg,pbut])
            wlist.append(top)
        for key,w in self._floatWidgets(text):
            self.widgets[key] = w
            if properties:
                self.properties.add(key)
            wlist.append(w)
        self.widget_list.extend(wlist)
        
    def show(self,text):
        dshow(text,data=self.data)

In [7]:
w = What()
w.defWidgets('Fy=345.,Fu=450.')
w.defWidgets('Zx,Sx,Ix,Iy,J,Cw,D,B,T,W',properties=True)
w.defWidgets('L,M1,M2')

In [10]:
w.interact(lambda: w.show('Fy,Fu'))

Output(layout=Layout(border='1px solid black'))

In [9]:
w.data

{'Fy': 345.0,
 'Fu': 450.0,
 'Zx': 0.0,
 'Sx': 0.0,
 'Ix': 0.0,
 'Iy': 0.0,
 'J': 0.0,
 'Cw': 0.0,
 'D': 0.0,
 'B': 0.0,
 'T': 0.0,
 'W': 0.0,
 'L': 0.0,
 'M1': 0.0,
 'M2': 0.0}

In [2]:
SECT = None   # holds the properties of one shape load from the steel tables
WIDG = {}     # holds all the value widgets, indexed by property name

def _makeFloat(key):
    w = widgets.FloatText( value=0.0, description=key+':')
    global WIDG
    WIDG[key] = w
    return w

inputs = [_makeFloat(x.strip()) for x in 'Sx,Zx,Ix,Iy,J,Cw,D,B,T,W'.split(',')]

output = widgets.Output(layout={'border': '1px solid black'})

In [3]:
@output.capture(clear_output=True)
def on_compute_clicked(b):
    global SECT
    global WIDG
    for k,w in WIDG.items():
        if hasattr(SECT,k):
            setattr(SECT,k,w.value)
    show('Dsg,*10^3,Sx,Zx,*10^6,Ix,Iy,*10^3,J,*10^9,Cw,*,D,B,T,W',object=SECT)

In [4]:
@output.capture(clear_output=True)
def on_load_clicked(b):
    global WIDG
    with output:
        Dsg = WIDG['Dsg'].value
        ##print('Dsg =',Dsg)
        global SECT
        SECT = SST.section(Dsg)
        for key,w in WIDG.items():  # propagate values to widgets
            if hasattr(SECT,key):
                w.value = getattr(SECT,key)
    ##output.clear_output()

In [5]:
WIDG['Dsg'] = widgets.Text(value='W250x80',
                           placeholder='Shape Designation',
                           description='Dsg:',
                          tooltip='A shape designation to lookup for values.')
lb = widgets.Button(description='Load Properties',
                   tooltip='Click to load properties into entries below.')
lb.on_click(on_load_clicked)
top = widgets.HBox([WIDG['Dsg'],lb])
b = widgets.Button(description='Compute!',tooltip='Click to perform calculations')
b.on_click(on_compute_clicked)
allwidg = widgets.VBox([top]+inputs+[b])
display(allwidg)
display(output)

Output(layout=Layout(border='1px solid black'))